In [ ]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

In [ ]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  \
pinecone-client \
pandas \
sentence-transformers

In [ ]:
from pinecone import Pinecone
import os
import panadas as pd
import numpy as np

import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

# Prepare Data - New Articles

# Cesate Embeddings and Vectors

In [ ]:
# average_word_embeddings_komninos

In [ ]:
# check dimensionality of embeddings

# Setupup Pinecone index
	- Make sure the dimenionality of the embeddings matches the dimensionality of the index

# Upsert vector data

# Query vector data

# Query with metadata filters

# Cleanup